In [1]:
import os
import matplotlib.pyplot as plt
import scipy.misc
import sklearn.feature_extraction
import numpy as np
import scipy.ndimage
from skimage import measure, io
from skimage import transform
import train_r2 
import skimage
import json
import datetime
import pickle


save_results = r'C:\Users\buggyr\Mosaic_Experiments\models'
load_training = r'C:\Users\buggyr\Mosaic_Experiments\data\external\Training Images 3'

# trainmospatches=np.load(os.path.join(load_training,'input_img_mos_patch.npy'))
# trainpatches=np.load(os.path.join(load_training,'input_img_patch.npy'))

# trainmospatches = (trainmospatches/255)*2 - 1
# trainpatches = (trainpatches/255)*2 - 1

Using TensorFlow backend.


In [2]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D, Activation, UpSampling2D,Reshape
from keras.optimizers import Adadelta, Nadam, RMSprop
from keras.models import load_model

In [3]:
f1 = 64
f2 = 32
f3 = 32
ff = 3
k1 = (9,9)
k2 = (1,1)
k3 = (5,5)

model = Sequential()

model.add(Conv2D(f1,k1, padding = 'same', input_shape=(None, None ,4), activation = 'relu'))

model.add(UpSampling2D(size=(2,2)))

model.add(Conv2D(f2,k2, padding = 'same', activation='relu'))

model.add(Conv2D(f3,k3, padding = 'same',activation = 'relu'))

model.add(Conv2D(ff,k3, padding = 'same',activation = 'tanh'))


#model.add(Reshape((64,64,3)))
modsum = model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, None, None, 64)    20800     
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, None, None, 64)    0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, None, None, 32)    2080      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, None, None, 32)    25632     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, None, None, 3)     2403      
Total params: 50,915
Trainable params: 50,915
Non-trainable params: 0
_________________________________________________________________


In [4]:
f1 = 64
f2 = 32
f3 = 32
ff = 3
k1 = (9,9)
k2 = (1,1)
k3 = (5,5)

model = Sequential()

model.add(Conv2D(f1,k1, padding = 'same', input_shape=(None, None ,4), activation = 'relu'))

model.add(UpSampling2D(size=(2,2)))

model.add(Conv2D(f2,k2, padding = 'same', activation='relu'))

model.add(Conv2D(f3,k3, padding = 'same',activation = 'relu'))

model.add(Conv2D(ff,k3, padding = 'same',activation = 'tanh'))

model.add(Conv2D(ff,1, padding = 'same', kernel_initializer=train_r2.yuv_init, 
                 input_shape = (None, None, 3), trainable = False))

#model.add(Reshape((64,64,3)))
modsum = model.summary()




_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, None, None, 64)    20800     
_________________________________________________________________
up_sampling2d_2 (UpSampling2 (None, None, None, 64)    0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, None, None, 32)    2080      
_________________________________________________________________
conv2d_7 (Conv2D)            (None, None, None, 32)    25632     
_________________________________________________________________
conv2d_8 (Conv2D)            (None, None, None, 3)     2403      
_________________________________________________________________
conv2d_9 (Conv2D)            (None, None, None, 3)     12        
Total params: 50,927
Trainable params: 50,915
Non-trainable params: 12
_________________________________________________________________


In [25]:
now=datetime.datetime.now()
save_file=os.path.join(save_results,now.strftime("%Y-%m-%d %H-%M"))
os.mkdir(save_file)
save_pred = os.path.join(save_file,'Epoch_Predictions')
os.mkdir(save_pred)

with open(os.path.join(save_file,'Model_Summary.txt'),'w') as fh:
    # Pass the file handle in as a lambda function to make it callable
    model.summary(print_fn=lambda x: fh.write(x + '\n'))
    
#rmsprop = RMSprop(lr=0.0005, rho=0.9, epsilon=1e-08, decay=0.0)
optimizer_func = Nadam(lr=0.00002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, schedule_decay=0.004)
loss_func='mse'

early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)
tbCallBack = keras.callbacks.TensorBoard(log_dir=os.path.join(save_file,'TNSR_BRD'), histogram_freq=0, write_graph=True, write_images=True)
csv_logger = keras.callbacks.CSVLogger(os.path.join(save_file,'training.log'), separator=',', append=False)
epoch_predict = train_r2.Save_predictions(save_pred)


model.compile(optimizer=optimizer_func,loss=loss_func)

In [ ]:
#train_r.train_seq(load_training,32,40)
fls = len(os.listdir(load_training))

                            #train_dir, patch_size, batch_size
train_generator = train_r2.train_generator_yuv(load_training,32,32)
history = model.fit_generator(generator = train_generator,steps_per_epoch=3*fls,
                             epochs = 200,callbacks = [tbCallBack,csv_logger,epoch_predict])
print(history.history)

model.save(os.path.join(save_file,'DeMos_mod.h5'))

Epoch 1/200
4106/4107 [============================>.] - ETA: 0s - loss: 3.9962e-04

C:\ProgramData\Anaconda3\lib\site-packages\skimage\util\dtype.py:122: UserWarning: Possible precision loss when converting from float32 to uint8
  .format(dtypeobj_in, dtypeobj_out))


4107/4107 [==============================] - 180s - loss: 4.0000e-04   
Epoch 2/200
4107/4107 [==============================] - 180s - loss: 3.7576e-04   
Epoch 3/200
4107/4107 [==============================] - 186s - loss: 3.5979e-04   
Epoch 4/200
4107/4107 [==============================] - 183s - loss: 3.4732e-04   
Epoch 5/200
4107/4107 [==============================] - 178s - loss: 3.3449e-04   
Epoch 6/200
4107/4107 [==============================] - 178s - loss: 3.2310e-04   
Epoch 7/200
4107/4107 [==============================] - 179s - loss: 3.1325e-04   
Epoch 8/200
4107/4107 [==============================] - 179s - loss: 3.0269e-04   
Epoch 9/200
4107/4107 [==============================] - 182s - loss: 2.9241e-04   
Epoch 10/200
4107/4107 [==============================] - 180s - loss: 2.9012e-04   
Epoch 11/200
4107/4107 [==============================] - 179s - loss: 2.8108e-04   
Epoch 12/200
4107/4107 [==============================] - 180s - loss: 2.7730e-04   
E

In [ ]:
ptch_sizes = [32, 64, 128]
for ptch_size in ptch_sizes:
    print(str(patch_size))
    
    now=datetime.datetime.now()
    save_file=os.path.join(save_results,now.strftime("%Y-%m-%d %H-%M"))
    os.mkdir(save_file)
    save_pred = os.path.join(save_file,'Epoch_Predictions')
    os.mkdir(save_pred)

    with open(os.path.join(save_file,'Model_Summary.txt'),'w') as fh:
        # Pass the file handle in as a lambda function to make it callable
        model.summary(print_fn=lambda x: fh.write(x + '\n'))

    #rmsprop = RMSprop(lr=0.0005, rho=0.9, epsilon=1e-08, decay=0.0)
    optimizer_func = Nadam(lr=0.00002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, schedule_decay=0.004)
    loss_func='mse'

    early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)
    tbCallBack = keras.callbacks.TensorBoard(log_dir=os.path.join(save_file,'TNSR_BRD'), histogram_freq=0, write_graph=True, write_images=True)
    csv_logger = keras.callbacks.CSVLogger(os.path.join(save_file,'training.log'), separator=',', append=False)
    epoch_predict = train_r2.Save_predictions(save_pred)


    model.compile(optimizer=optimizer_func,loss=loss_func)

    #train_r.train_seq(load_training,32,40)
    fls = len(os.listdir(load_training))

                               #train_dir, patch_size, batch_size
    train_generator = train_r2.train_generator2(load_training,patch_size,32)
    history = model.fit_generator(generator = train_generator,steps_per_epoch=3*fls,
                                 epochs = 100,callbacks = [tbCallBack,csv_logger,epoch_predict])

    model.save(os.path.join(save_file,'DeMos_mod.h5'))
    

Epoch 1/100
4106/4107 [============================>.] - ETA: 0s - loss: 0.0069

C:\ProgramData\Anaconda3\lib\site-packages\skimage\util\dtype.py:122: UserWarning: Possible precision loss when converting from float32 to uint8
  .format(dtypeobj_in, dtypeobj_out))


4107/4107 [==============================] - 147s - loss: 0.0069   
Epoch 2/100
4107/4107 [==============================] - 174s - loss: 0.0049   
Epoch 3/100
4107/4107 [==============================] - 143s - loss: 0.0039   
Epoch 4/100
4107/4107 [==============================] - 146s - loss: 0.0032   
Epoch 5/100
4107/4107 [==============================] - 144s - loss: 0.0027   
Epoch 6/100
4107/4107 [==============================] - 142s - loss: 0.0023   
Epoch 7/100
4107/4107 [==============================] - 142s - loss: 0.0020   
Epoch 8/100
4107/4107 [==============================] - 142s - loss: 0.0018   
Epoch 9/100
4107/4107 [==============================] - 143s - loss: 0.0016   
Epoch 10/100
4107/4107 [==============================] - 142s - loss: 0.0015   
Epoch 11/100
4107/4107 [==============================] - 145s - loss: 0.0014   
Epoch 12/100
4107/4107 [==============================] - 142s - loss: 0.0013   
Epoch 13/100
4107/4107 [=========================

In [ ]:
i6=np.load(r'C:\Users\buggyr\Mosaic_Experiments\data\processed\Kodak_test\mos_npy\kodim19.npy')
g=i6[:,:,0]
b=i6[:,:,1]
r=i6[:,:,3]
i62=np.stack((r,g,b),2)
i62=i62.astype(np.float32)
plt.imshow(i62)
plt.show()
pred1 = model.predict(np.array([i6]), batch_size=1)[0]
print(pred1.shape)
i7 = abs(pred1)
plt.imshow(i7)
plt.show()

i8=np.load(r'C:\Users\buggyr\Mosaic_Experiments\data\processed\Kodak_test\img_npy\kodim19.npy')
plt.imshow(i8)
plt.show()

i62=(transform.rescale(i62,(2,2))).astype(np.float32) 

slh=os.path.join(save_file,'lighthouse_images')
os.mkdir(slh)

skimage.io.imsave(os.path.join(slh,'input mosaiced.png'),i62)
skimage.io.imsave(os.path.join(slh,'predicted.png'),i7)
skimage.io.imsave(os.path.join(slh,'original.png'),i8)

psnr_mos = measure.compare_psnr(i62,i8)
print('Mosaiced and Interpolated PSNR: '+str(psnr_mos))

psnr_pred = measure.compare_psnr(i7,i8)
print('Predicted PSNR: '+str(psnr_pred))

ssim= measure.compare_ssim(i7,i8,multichannel=True)
print('Predicted SSIM: '+str(ssim))

In [ ]:
i6=np.load(r'C:\Users\buggyr\Mosaic_Experiments\data\processed\Kodak_test\mos_npy\kodim19.npy')
g=i6[:,:,0]
b=i6[:,:,1]
r=i6[:,:,3]
i62=np.stack((r,g,b),2)
i62=i62.astype(np.float32)
plt.imshow(i62)
plt.show()
pred1 = model.predict(np.array([i6]), batch_size=1)[0]
print(pred1.shape)
i7 = abs(pred1)
plt.imshow(i7)
plt.show()

i8=np.load(r'C:\Users\buggyr\Mosaic_Experiments\data\processed\Kodak_test\img_npy\kodim19.npy')
plt.imshow(i8)
plt.show()

i62=(transform.rescale(i62,(2,2))).astype(np.float32) 

slh=os.path.join(save_file,'lighthouse_images')
os.mkdir(slh)

skimage.io.imsave(os.path.join(slh,'input mosaiced.png'),i62)
skimage.io.imsave(os.path.join(slh,'predicted.png'),i7)
skimage.io.imsave(os.path.join(slh,'original.png'),i8)

psnr_mos = measure.compare_psnr(i62,i8)
print('Mosaiced and Interpolated PSNR: '+str(psnr_mos))

psnr_pred = measure.compare_psnr(i7,i8)
print('Predicted PSNR: '+str(psnr_pred))

ssim= measure.compare_ssim(i7,i8,multichannel=True)
print('Predicted SSIM: '+str(ssim))

In [ ]:
import pickle
k_orig = r'C:\Users\buggyr\Mosaic_Experiments\data\processed\Kodak_test\img_lst'
k_mos = r'C:\Users\buggyr\Mosaic_Experiments\data\processed\Kodak_test\mos_lst'

with open(k_orig, "rb") as fp1:   # Unpickling
    k_or_imgs = pickle.load(fp1)
    
with open(k_mos, "rb") as fp2:   # Unpickling
    k_mos_imgs = pickle.load(fp2)

psnr = 0
ssim = 0
smpls=len(k_mos_imgs)
for i in range(0,smpls):
    w = k_or_imgs[i].shape[0]
    h = k_or_imgs[i].shape[1]
    pred_imgs = (model.predict(np.array([k_mos_imgs[i]]),batch_size=1)).astype(np.uint8)
    orig_imgs = (k_or_imgs[i]).astype(np.uint8) 
    
    psnr = psnr + measure.compare_psnr(pred_imgs[i,5:h-5,5:w-5,:],orig_imgs[5:h-5,5:w-5,:])
    ssim= ssim + measure.compare_ssim(pred_imgs[i,5:h-5,5:w-5,:],orig_imgs[5:h-5,5:w-5,:],multichannel=True)
psnr_K_avg = psnr/smpls
ssim_K_avg = ssim/smpls
print('Kodak PSNR Average: '+str(psnr_K_avg))
print('Kodak SSIM Average: '+str(ssim_K_avg))

In [ ]:
pred_imgs = (model.predict(np.load(r'C:\Users\buggyr\Mosaic_Experiments\data\processed\McM_test\mos_npy\sv.npy'), batch_size=1)).astype(np.uint8)
orig_imgs = (np.load(r'C:\Users\buggyr\Mosaic_Experiments\data\processed\McM_test\img_npy\sv.npy')).astype(np.uint8)
psnr=0
smpls=orig_imgs.shape[0]
for i in range(0,smpls):
    psnr = psnr + measure.compare_psnr(pred_imgs[i,:,:,:],orig_imgs[i,:,:,:])
    ssim= ssim + measure.compare_ssim(pred_imgs[i,:,:,:],orig_imgs[i,:,:,:],multichannel=True)
    
psnr_McM_avg = psnr/smpls
ssim_McM_avg = ssim/smpls    
print('McM PSNR Average: '+str(psnr/smpls))
print('McM SSIM Average: '+str(ssim_McM_avg))

In [ ]:
data = {}
data['Results'] = {
    'McM PSNR':str(psnr_McM_avg),
    'McM SSIM':str(ssim_McM_avg),
    'Kodak PSNR':str(psnr_K_avg),
    'Kodak SSIM':str(ssim_K_avg),
}
data['Parameters'] = {
    'Loss Function': loss_func,
    'Optimizer':str(type(optimizer_func))
}
data['Training Set'] = {
    'Training Path': load_training,
}

with open(os.path.join(save_file,'results.txt'), 'w') as outfile:  
    json.dump(data, outfile,indent=0)

In [23]:
import importlib
importlib.reload(train_r2)



<module 'train_r2' from 'C:\\Users\\buggyr\\Mosaic_Experiments\\src\\train_r2.py'>

In [24]:
import train_r2

train_generator_yuv = train_r2.train_generator_yuv(load_training,128,32)

%prun next(train_generator_yuv)

In [ ]:

import cv2
import numpy as np

im = cv2.imread(r'C:\Users\buggyr\Mosaic_Experiments\data\external\Data_misc\cat.jpg')
img_ptch = train_r2.extractPatches(im,128) 
mos,orig = train_r2.mosaic(img_ptch)


mos.shape

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
plt.imshow(cv2.cvtColor(orig[0], cv2.COLOR_BGR2RGB))
plt.show()
plt.imshow(cv2.cvtColor(mos[0,:,:,1:], cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
kodak_pth = r'C:\Users\buggyr\Mosaic_Experiments\data\interim\Kodak'
predict_gen = train_r2.predict_generator2(kodak_pth)
predk = predict_generator(predict_gen,steps = 24)
orig = np.load(r'C:\Users\buggyr\Mosaic_Experiments\data\processed\Kodak_test\img_npy')
for i in range(predk[0]):
    
    psnr = psnr + measure.compare_psnr(pred_k[i,:,:,:],orig_imgs)
    ssim= ssim + measure.compare_ssim(pred_imgs[0,:,:,:],orig_imgs,multichannel=True)
